In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import pandas as pd
import torch
import glob
import pickle
import matplotlib.pyplot as plt

from interpretation.interpret import compute_importance_score_c_type, compute_importance_score_bias, visualize_sequence_imp
from models.models import CATAC2, CATAC_w_bias

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Sample from peak sequences

In [6]:
with open('../results/peaks_seq.pkl', 'rb') as file:
    seq = pickle.load(file)

seq = seq[seq.chr.isin(['6','13', '22'])].sequence
seq

,chr,start,end,middle_peak,sequence,GC_cont
peakID,,,,,,
13:18211588-18212586,13,18211588,18212586,18212087,AAGACTCTGAGAAAAACTGTAAATTAAACACAAATATGGTGGAAGC...,0.347412
13:18212943-18213672,13,18212943,18213672,18213308,GGTACATACAGACCATTGAATACTATGCAGCCACAAAAAAGAATAA...,0.295654
13:19587105-19588284,13,19587105,19588284,19587694,AAGGAAGTTATACAAAAAAACAATTTTGGGTGAGGTTGTTATCTAG...,0.452148
13:19597289-19598094,13,19597289,19598094,19597692,CAAACAGTCACCTGGAGCAGAGAGGTCAGTGTGTTTACGTGTTTCC...,0.466309
13:19601665-19602525,13,19601665,19602525,19602095,TTAGTAGAGAAGGGGTTTCACCATGTGTTGGTCAGGCTGGTCTCCA...,0.529541
...,...,...,...,...,...,...
6:170553912-170554797,6,170553912,170554797,170554354,TTCAATTTACAGCTCTTCCCTGTCAAGAGTCTTAAACAGAGCATCT...,0.453369
6:170574897-170575869,6,170574897,170575869,170575383,TGTAAAAGTACATCTTCAGCTGACTCAGGAATAAAATCAGAAAGGG...,0.375244
6:170584150-170585077,6,170584150,170585077,170584614,TGCTGGATAAATGTTGGCTACTATAATAAAATAAGCCTCTAAGATA...,0.483154


# Add sequence overlapping with CUT&Tag peaks

In [ ]:
""" #Open peak pandas dataframe and write to .bed file
with open('../results/peaks_seq.pkl', 'rb') as file:
    peaks = pickle.load(file)

peaks.chr = ["chr" + str(c) for c in peaks.chr]
peaks[["chr","start","end"]].to_csv('../results/all_peaks.bed', sep="\t", index=False, header=False)

#Compute intersection between genomic regions
#-----------------------------------------------

#MyoD1
!bedtools intersect -a ../results/all_peaks.bed -b ../data/MYOD1_25m_s450.bed -wa > ../results/MyoD1_intersect.bed
MYOD1 = pd.read_csv("../results/MyoD1_intersect.bed", header=None, sep= "\t")
MYOD1["ID"] = MYOD1[0] + ":" +  MYOD1[1].astype('str') + "-" + MYOD1[2].astype("str")
MYOD1["ID"] =  [x[3:] for x in MYOD1.ID]; MYOD1 = MYOD1.ID.tolist()

#HES1
!bedtools intersect -a ../results/all_peaks.bed -b ../data/HES1_D8_REP1_filt_TFBS_HES1_s400.bed -wa > ../results/HES1_intersect.bed

HES1 = pd.read_csv("../results/HES1_intersect.bed", header=None, sep= "\t")
HES1["ID"] = HES1[0] + ":" +  HES1[1].astype('str') + "-" + HES1[2].astype("str")
HES1["ID"] =  [x[3:] for x in HES1.ID]; HES1 = HES1.ID.tolist()

#PAX3
!bedtools intersect -a ../results/all_peaks.bed -b ../data/PAX3_D8_REP1_s500.bed  -wa > ../results/PAX3_intersect.bed

PAX3 = pd.read_csv("../results/PAX3_intersect.bed", header=None, sep= "\t")
PAX3["ID"] = PAX3[0] + ":" +  PAX3[1].astype('str') + "-" + PAX3[2].astype("str")
PAX3["ID"] =  [x[3:] for x in PAX3.ID]; PAX3 = PAX3.ID.tolist()

TF_peaks = MYOD1 + HES1 + PAX3 """

/Applications/anaconda3/envs/LeKira/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
/Applications/anaconda3/envs/LeKira/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
/Applications/anaconda3/envs/LeKira/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [ ]:
""" extra = seq.loc[TF_peaks].sequence.tolist()
seq = seq.sample(10000).sequence.tolist() + extra
len(seq) """

12216

In [ ]:
""" seq = pd.Series(seq)
seq """

0        GAGGGACTTAGAACATGAGGGACCATCATCTCTGTTCAAATTCACT...
1        ACCCTGGGTGGGGATCCTCGGGGCTTCCGGGTGCAGACCTCCCCAC...
2        TGTTTTCTCCTCTGGAAAGGAGCATGCAGGTGTGTCTGGCTGAGAC...
3        AAAAAAAAAAAGGAACAGTGCTAGAGACAAGTTCAGATAACATCTT...
4        TTCTCCTGCCTCAGCCTCCCAAGTAGCAGGAATTACAGACCTGCAC...
                               ...                        
12211    GGAACTGTTATCATGTTAGAGTAAATTAGATTTCTTGAGGGAAGTG...
12212    GGAACTGTTATCATGTTAGAGTAAATTAGATTTCTTGAGGGAAGTG...
12213    GGAACTGTTATCATGTTAGAGTAAATTAGATTTCTTGAGGGAAGTG...
12214    GGAACTGTTATCATGTTAGAGTAAATTAGATTTCTTGAGGGAAGTG...
12215    TGGAAATATTTCATTGCTTGATAGTGGTACAAGTTAATGATTATGT...
Length: 12216, dtype: object

# Compute importance scores

In [ ]:
path_model = '../results/train_res/128_10_model.pkl'

all_c_type = ['Immature', 'Mesenchymal', 'Myoblast', 'Myogenic', 'Neuroblast',
       'Neuronal', 'Somite']
time_point = ["D8", "D12", "D20", "D22"]

first_kernel = 21
nb_conv = 10
size_final_conv = 4096 - (first_kernel - 1)
cropped = [2**l for l in range(0,nb_conv-1)] * (2*(3-1))

for c in cropped:
       size_final_conv -= c

#Load the model
model = CATAC_w_bias(nb_conv=10, nb_filters=128, first_kernel=21, 
                      rest_kernel=3, out_pred_len=1024, 
                      nb_pred=4, size_final_conv=size_final_conv)
        
model.load_state_dict(torch.load(path_model, map_location=torch.device('cpu')))

path_model_bias = "../data/Tn5_NN_model.h5"

#Compute attribution scores
seq, shap_scores, proj_scores = compute_importance_score_bias(model, path_model_bias, seq, device, "Myogenic", all_c_type, 1)

In [ ]:
#Save encoded seq + scores
np.savez('../results/encod_seq.npz', seq[:,:4,:])
np.savez('../results/seq_scores.npz', shap_scores[:,:4,:], proj_scores[:,:4,:])

print("Shap scores saved!")

In [ ]:
seq = np.load('../results/encod_seq.npz')["arr_0"]
shap_scores = np.load('../results/seq_scores.npz')
proj_scores = shap_scores['arr_1']; shap_scores = shap_scores['arr_0']

# Visualize few examples

In [ ]:
visualize_sequence_imp(proj_scores[[73],:4,:] ,0, 4096)
visualize_sequence_imp(proj_scores[[1266],:4,:] ,0, 4096)
visualize_sequence_imp(proj_scores[[563],:4,:] ,0, 4096)

# Use TF-modisco to find TFBS
Following tutorial at: https://github.com/jmschrei/tfmodisco-lite/blob/main/examples/ModiscoDemonstration.ipynb

In [ ]:
!modisco motifs -s  ../results/encod_seq.npz -a  ../results/seq_scores.npz -n 2000 -o modisco_results.h5

In [ ]:
!modisco report -i modisco_results.h5 -o report/

In [ ]:
from IPython.display import HTML
HTML('report/motifs.html')

# Run TOMTOM on modisco results

In [ ]:
!modisco report -i modisco_results.h5 -o report/TOMTOM/ -s report/TOMTOM/ -m ../data/JASPAR_motif.txt

In [ ]:
from IPython.display import HTML

HTML('report/motifs.html')